In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from src import object_test

# Fighter page

In [2]:
fighter_link = '/fightcenter/fighters/51080-curtis-blaydes'

In [3]:
fighter_page = object_test.open_tapology_link(fighter_link)
fighter_soup = BeautifulSoup(fighter_page)

#### Get list of info
The data I want is in the table under the element with the class 'details details_two_columns'. I can pull this in with my html_list_to_df function.

In [11]:
info = object_test.html_li_to_df(fighter_soup, 'details details_two_columns')
info

,header,ProMMARecord,Nickname,CurrentStreak,Age,LastFight,WeightClass,Affiliation,Height,CareerDisclosedEarnings,Born,Fightingoutof,College,FoundationStyle,FighterLinks,PersonalLinks
0,Name,"14-2-0, 1 NC (Win-Loss-Draw)",Razor,4 Wins,29\n| Date of Birth,"June 20, 2020\nin\nUFC",Heavyweight\n| Last Weigh-In,Elevation Fight Team,"6'4"" (193cm)\n| Reach","$338,000 USD","Naperville, Illinois","Chicago, Illinois",Harper College,Wrestling,,


It's missing reach, so let's see if we can get it back in there. Heres the code for the function i wrote.

In [22]:
list_class = 'details details_two_columns'

### Find the list element

div = fighter_soup.find(class_=list_class) #grab top header
event_info_elem = div.find('ul') #grab first list in top header

### Find the list items in the element

html_list_items = event_info_elem.find_all('li')
html_list_items = [item.get_text() for item in html_list_items]
html_list_items

['\nName:\nCurtis Blaydes\n',
 '\nPro MMA Record:\n14-2-0, 1 NC (Win-Loss-Draw)\n',
 '\nNickname:\nRazor\n',
 '\nCurrent Streak:\n4 Wins\n',
 '\nAge:\n29\n| Date of Birth:\n1991.02.18\n',
 '\nLast Fight:\nJune 20, 2020\nin\nUFC\n',
 '\nWeight Class:\nHeavyweight\n| Last Weigh-In:\n261.0 lbs\n',
 '\nAffiliation:\nElevation Fight Team\n',
 '\nHeight:\n6\'4" (193cm)\n| Reach:\n80.0" (203cm)\n',
 '\nCareer Disclosed Earnings:\n$338,000 USD\n',
 '\nBorn:\nNaperville, Illinois\n',
 '\nFighting out of:\nChicago, Illinois\n',
 '\nCollege:\nHarper College\n',
 '\nFoundation Style:\nWrestling\n',
 '\nFighter Links:\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n',
 '\nPersonal Links:\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n']

In [24]:
column_value_strings = []
for row in html_list_items:
    if '\n|' in row:
        column_value_strings+=row.split('\n|')
    else:
        column_value_strings.append(row)
        
column_value_strings

['\nName:\nCurtis Blaydes\n',
 '\nPro MMA Record:\n14-2-0, 1 NC (Win-Loss-Draw)\n',
 '\nNickname:\nRazor\n',
 '\nCurrent Streak:\n4 Wins\n',
 '\nAge:\n29',
 ' Date of Birth:\n1991.02.18\n',
 '\nLast Fight:\nJune 20, 2020\nin\nUFC\n',
 '\nWeight Class:\nHeavyweight',
 ' Last Weigh-In:\n261.0 lbs\n',
 '\nAffiliation:\nElevation Fight Team\n',
 '\nHeight:\n6\'4" (193cm)',
 ' Reach:\n80.0" (203cm)\n',
 '\nCareer Disclosed Earnings:\n$338,000 USD\n',
 '\nBorn:\nNaperville, Illinois\n',
 '\nFighting out of:\nChicago, Illinois\n',
 '\nCollege:\nHarper College\n',
 '\nFoundation Style:\nWrestling\n',
 '\nFighter Links:\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n',
 '\nPersonal Links:\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n']

In [26]:
### Insert a 'header' label.
#### The info in the header may be useful later on

item_list = [item.split(':\n') for item in column_value_strings]
item_list[0].insert(0, 'header')
item_list

### Encapsulate the second element of each row in a list
#### This allows them to be made into a dataframe easily

item_list = [[item[0], [item[1]]] for item in item_list]
item_list

### Convert to dataframe

list_df = pd.DataFrame(dict(item_list))

# clean up white space
list_df = list_df.applymap(lambda x: x.strip())
list_df.columns = [(col.strip()).replace(' ', '') for col in list_df.columns]

In [27]:
list_df

,header,ProMMARecord,Nickname,CurrentStreak,Age,DateofBirth,LastFight,WeightClass,LastWeigh-In,Affiliation,Height,Reach,CareerDisclosedEarnings,Born,Fightingoutof,College,FoundationStyle,FighterLinks,PersonalLinks
0,Name,"14-2-0, 1 NC (Win-Loss-Draw)",Razor,4 Wins,29,1991.02.18,"June 20, 2020\nin\nUFC",Heavyweight,261.0 lbs,Elevation Fight Team,"6'4"" (193cm)","80.0"" (203cm)","$338,000 USD","Naperville, Illinois","Chicago, Illinois",Harper College,Wrestling,,


In [28]:
object_test.html_li_to_df(fighter_soup, 'details details_two_columns')

,header,ProMMARecord,Nickname,CurrentStreak,Age,DateofBirth,LastFight,WeightClass,LastWeigh-In,Affiliation,Height,Reach,CareerDisclosedEarnings,Born,Fightingoutof,College,FoundationStyle,FighterLinks,PersonalLinks
0,Name,"14-2-0, 1 NC (Win-Loss-Draw)",Razor,4 Wins,29,1991.02.18,"June 20, 2020\nin\nUFC",Heavyweight,261.0 lbs,Elevation Fight Team,"6'4"" (193cm)","80.0"" (203cm)","$338,000 USD","Naperville, Illinois","Chicago, Illinois",Harper College,Wrestling,,
